In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [27]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,37.06,87,40,6.91,AR,1621665562
1,1,pevek,69.7008,170.3133,30.09,90,91,6.98,RU,1621665856
2,2,doka,13.5167,35.7667,98.58,8,9,11.90,SD,1621665856
3,3,cape town,-33.9258,18.4232,52.12,60,0,3.20,ZA,1621665670
4,4,mataura,-46.1927,168.8643,49.48,93,0,2.17,NZ,1621665857


In [28]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [29]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,61,sinazongwe,-17.2614,27.4618,71.17,49,0,2.37,ZM,1621665879
209,209,vila velha,-20.3297,-40.2925,71.94,82,0,4.00,BR,1621665941
387,387,morondava,-20.2833,44.2833,78.17,43,0,7.11,MG,1621666013
420,420,adeje,28.1227,-16.7260,75.02,60,0,1.99,ES,1621666027


In [31]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
61,sinazongwe,ZM,-17.2614,27.4618,
209,vila velha,BR,-20.3297,-40.2925,
387,morondava,MG,-20.2833,44.2833,
420,adeje,ES,28.1227,-16.7260,


In [32]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
# Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,Country,Lat,Lng,Hotel Name
61,sinazongwe,ZM,-17.2614,27.4618,Lakeview Lodge
209,vila velha,BR,-20.3297,-40.2925,Quality Suites Vila Velha
387,morondava,MG,-20.2833,44.2833,La Case Bambou
420,adeje,ES,28.1227,-16.7260,Gran Tacande Wellness & Relax


In [33]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))